In [3]:
import boto3
from botocore.client import Config
import os
from pathlib import Path
import uuid

In [4]:
# Parámetros de conexión a MinIO
minio_endpoint = "minio:9000"
minio_access_key = "admin"
minio_secret_key = "admin123"

# Cliente S3 (boto3)
s3 = boto3.client(
    's3',
    endpoint_url=f"http://{minio_endpoint}",
    aws_access_key_id=minio_access_key,
    aws_secret_access_key=minio_secret_key,
    config=Config(signature_version='s3v4'),
    region_name='us-east-1'
)

In [5]:
# Crear bucket (si no existe)
bucket_name = "mis-pruebas"

existing_buckets = [b['Name'] for b in s3.list_buckets()['Buckets']]

if bucket_name not in existing_buckets:
    s3.create_bucket(Bucket=bucket_name)
    print(f"Bucket '{bucket_name}' creado.")
else:
    print(f"Bucket '{bucket_name}' ya existe.")

Bucket 'mis-pruebas' creado.


In [6]:
# Crear archivo de prueba
local_file = Path("/home/jovyan/work/test_file.txt")
local_file.write_text("Este es un archivo de prueba generado por Jupyter y subido a MinIO.")

# Subir archivo al bucket
object_key = f"archivo-{uuid.uuid4()}.txt"
s3.upload_file(str(local_file), bucket_name, object_key)
print(f"Archivo subido a '{bucket_name}/{object_key}'")

Archivo subido a 'mis-pruebas/archivo-ad44c08d-22f5-4294-9d5d-c927ca8cfc7f.txt'


In [7]:
# Listar objetos en el bucket
response = s3.list_objects_v2(Bucket=bucket_name)

if 'Contents' in response:
    print("Archivos en el bucket:")
    for obj in response['Contents']:
        print(" -", obj['Key'])
else:
    print("No hay archivos en el bucket.")

Archivos en el bucket:
 - archivo-ad44c08d-22f5-4294-9d5d-c927ca8cfc7f.txt
